# Proyecto 04 - Informe Final Carrera

***Presentado por:*** *Sandra Marcela Balbin Agudelo*
***Grupo:*** *DS-ONLINE-38*

## Sistema de recomendación basado en contenidos.

Este proyecto, así como el proyecto 3, se pueden ver y descargar [aquí](https://github.com/smbalbin/Data-Science.git).

**Objetivo:** Desarrollar un sistema de recomendación basado en contenidos, para recomendar juegos a los usuarios de acuerdo a los géneros a los que suelen recurrir.

Los sistemas de recomendación Content-Based son algoritmos utilizados para sugerir elementos a los usuarios basados en sus preferencias. Se parte de información del mismo usuario para recomendar otras opciones similares a las favoritas del mismo. Estos recomendadores son muy comunes en aplicaciones de contenido como Netflix o Spotify, así como en tiendas online y buscadores de trabajo.

Este tipo de modelo es aplicable al dataset de juegos disponible, y puede brindar una recomendación más exacta que la ofrecida por el filtro colaborativo.

Para cumplir el objetivo se implementará una versión simple de este tipo de recomendadores, usando pandas.

* **Importación de librerías**

In [1]:
import numpy as np
import pandas as pd
import gzip
from math import sqrt

pd.set_option('display.width', 150)

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

* **Carga de los dataset y visualización inicial**

***Dataset reviews***

In [2]:
contador = 0
data_reviews = []
n = 10
for l in parse('steam_reviews.json.gz'):
    if contador%n == 0:
        data_reviews.append(l)
    else:
        pass
    contador += 1

In [3]:
data_reviews = pd.DataFrame(data_reviews)

In [4]:
data_reviews.head(5)

,username,hours,products,product_id,page_order,date,text,early_access,page,user_id,compensation,found_funny
0,Chaos Syren,0.1,41.0,725280,0,2017-12-17,This would not be acceptable as an entertainme...,False,1,NaN,NaN,NaN
1,Ariman1,13.2,1386.0,328100,2,2017-08-02,Addictive RPG ! Works fine on linux though it ...,False,1,NaN,NaN,NaN
2,freakfantom,0.1,1706.0,725280,5,2017-11-12,Прикольная стрелялка. Взял дешево на распродаже.,False,1,NaN,NaN,NaN
3,The_Cpt_FROGGY,7.8,2217.0,631920,0,2017-12-11,Somewhere on Zibylon:\n~~~~~~~~~~~~~~~~~~\nZib...,False,1,76561198003030375,Product received for free,NaN
4,the_maker988,8.2,18.0,35140,7,2018-01-02,"This game was way to linear for me, and compar...",False,1,76561198353396844,NaN,NaN


In [5]:
data_reviews.shape

(779307, 12)

***Dataset games***

In [6]:
data_games = []
for l in parse('steam_games.json.gz'):
    data_games.append(l)
data_games = pd.DataFrame(data_games)

In [7]:
data_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,discount_price,reviews_url,specs,price,early_access,id,developer,sentiment,metascore
0,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",4.49,http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,False,761140,Kotoshiro,NaN,NaN
1,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",NaN,http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,False,643980,Secret Level SRL,Mostly Positive,NaN
2,Poolians.com,"[Casual, Free to Play, Indie, Simulation, Sports]",Real Pool 3D - Poolians,Real Pool 3D - Poolians,http://store.steampowered.com/app/670290/Real_...,2017-07-24,"[Free to Play, Simulation, Sports, Casual, Ind...",NaN,http://steamcommunity.com/app/670290/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free to Play,False,670290,Poolians.com,Mostly Positive,NaN
3,彼岸领域,"[Action, Adventure, Casual]",弹炸人2222,弹炸人2222,http://store.steampowered.com/app/767400/2222/,2017-12-07,"[Action, Adventure, Casual]",0.83,http://steamcommunity.com/app/767400/reviews/?...,[Single-player],0.99,False,767400,彼岸领域,NaN,NaN
4,NaN,NaN,Log Challenge,NaN,http://store.steampowered.com/app/773570/Log_C...,NaN,"[Action, Indie, Casual, Sports]",1.79,http://steamcommunity.com/app/773570/reviews/?...,"[Single-player, Full controller support, HTC V...",2.99,False,773570,NaN,NaN,NaN


In [8]:
data_games.shape

(32135, 16)

### Parte 1 - Preprocesamiento de los dataset

***Dataset reviews***

* Se genera una copia del dataset para realizar las trasnformaciones.

In [9]:
reviews_copy = data_reviews.copy()

* Se eliminan las variables que no aportan suficiente información sobre los juegos, de esta manera se eliminan las columnas *page_order, date, early_acces, page, compensation* ya que son informativas, la variable *found funny* está en un 85% vacía por lo cual también se retira del dataset.

In [10]:
reviews_copy.drop(['page_order','date','early_access','page','found_funny','compensation','user_id','products','text'],
                  axis=1, inplace=True)

* Los registros que no cuentan con horas de juego, no son útiles para desarrollar este sistema de recomendación, por lo tanto se procede a eliminarlos. Estos representan un 0.3% del dataset.

In [11]:
reviews_copy.isna().sum()

username         0
hours         2637
product_id       0
dtype: int64

In [12]:
reviews_copy.dropna(subset=['hours'], inplace=True)

In [13]:
reviews_copy.isna().sum()

username      0
hours         0
product_id    0
dtype: int64

* La variable *user_id* tiene 60% de valores faltantes, por esta razón se elimina y se crea un nuevo **id** para identificar los usuarios.

In [14]:
user_names = list(np.unique(reviews_copy.username))

In [15]:
user_encoded = np.arange(0,len(user_names))

In [16]:
reviews_copy['user_id'] = reviews_copy.username.map(dict(zip(user_names,user_encoded)))

* El Dataset Reviews no cuenta con una calificación explícita de los juegos y para crear un sistema de recomendación deben existir calificaciones. Se asume que si un jugador juega más de 5 horas un juego es porque le gusta, bajo este supuesto se crea una calificación binaria en la variable *'ratings'* donde si es inferior a 5 horas obtiene '0' y si es igual o mayor a 5 horas '1'.

In [17]:
def ratings(time):
    if time >= 5:
        return 1
    else:
        return 0

In [18]:
ratings = reviews_copy['hours'].apply(ratings)

reviews_copy['ratings'] = ratings

* Se eliminan los juegos que tienen menos de 25 jugadores, pues no son lo suficientemente populares para desarrollar un sistema de recomendación.

In [19]:
lista = list(reviews_copy.product_id.value_counts().index[reviews_copy.product_id.value_counts().values >= 25])

In [20]:
reviews_copy = reviews_copy[reviews_copy.product_id.isin(lista)]

In [21]:
reviews_copy = reviews_copy.reset_index(drop = True)

* Se eliminan los usuarios con menos de 5 juegos, ya que tienen pocos juegos y puede afectar la eficiencia del modelo para recomendar.

In [22]:
lista_users = list(reviews_copy.username.value_counts().index[reviews_copy.username.value_counts().values >= 5])

In [23]:
reviews_copy = reviews_copy[reviews_copy.username.isin(lista_users)]

In [24]:
reviews_copy = reviews_copy.reset_index(drop = True)

In [25]:
reviews_copy.head(5)

,username,hours,product_id,user_id,ratings
0,Puddle,1.2,308040,268687,0
1,Hanzo,0.2,8880,144378,0
2,MLGarbage,9.5,35140,207207,1
3,Lov3toPlay,1.5,308040,202761,0
4,unit73e,2.7,308040,508319,0


In [26]:
reviews_copy.shape

(99260, 5)

***Dataset games***

* Se genera una copia del dataset, para realizar las transformaciones.

In [27]:
games_copy = data_games.copy()

* El modelo se construirá basado en los géneros de los juegos. La variable *'tags'* también ofrece una clasificación de los juegos, sin embargo, contiene más de 300 opciones, por lo cual se vuelve complejo el modelado. Por lo tanto, se eliminan las variables que no se usarán para el desarrollo del modelo, dejando solo los identificadores de cada juego (*'id'* y *'title'*) y los géneros.

In [28]:
games_copy.drop(['tags','publisher','app_name','url','release_date','discount_price','reviews_url','specs','price','early_access','developer','sentiment','metascore'],
                  axis=1, inplace=True)

* Se eliminan los registros que no contengan la variable *'genres'* ya que en esta variable se basará el sistema de recomendación. Además se eliminan registros que no tengan título (*'title'*) o (*'id'*), porque no se pueden recomendar juegos que no sean identificables. Adicional se eliminarán juegos duplicados.

In [29]:
games_copy.isna().sum()

genres    3283
title     2050
id           2
dtype: int64

In [30]:
games_copy.dropna(subset=['genres','title','id'], inplace=True)
games_copy.drop_duplicates(subset=['id'], inplace=True)

In [31]:
games_copy.shape

(28849, 3)

* Se dividen los valores de la columna *'genres'* mediante encoding, asignando *'1'* a los juegos que tienen el género y *'0'* a los que no.

In [32]:
from sklearn.preprocessing import MultiLabelBinarizer
pd.options.mode.chained_assignment = None 

In [33]:
mlb = MultiLabelBinarizer()
genre_encoded = mlb.fit_transform(games_copy['genres'])
genre_encoded = pd.DataFrame(genre_encoded,columns=mlb.classes_).reset_index(drop = True)
genre_encoded = genre_encoded.astype(int)

In [34]:
games_copy = pd.concat([games_copy[['id','title','genres']], genre_encoded], axis=1)
games_copy.sort_index(inplace = True)
games_copy.head()

,id,title,genres,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,761140,Lost Summoner Kitty,"[Action, Casual, Indie, Simulation, Strategy]",0.0,1.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1,643980,Ironbound,"[Free to Play, Indie, RPG, Strategy]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,670290,Real Pool 3D - Poolians,"[Casual, Free to Play, Indie, Simulation, Sports]",0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
3,767400,弹炸人2222,"[Action, Adventure, Casual]",0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,0.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


### Parte 2 - Modelo de recomendación

* Para construir el modelo se debe generar un perfil de usuario tomando los '*ratings'* y los géneros. En este sentido, se adicionan los géneros al dataset reviews mediante un *inner join* para de paso excluir los juegos que no tengan géneros.

In [35]:
data_reviews1 = pd.merge(reviews_copy, games_copy, how='inner', left_on='product_id', right_on='id')
data_reviews1.head()

,username,hours,product_id,user_id,ratings,id,title,genres,Accounting,Action,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,Puddle,1.2,308040,268687,0,308040,Back to Bed,"[Action, Casual, Indie]",0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,Lov3toPlay,1.5,308040,202761,0,308040,Back to Bed,"[Action, Casual, Indie]",0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,unit73e,2.7,308040,508319,0,308040,Back to Bed,"[Action, Casual, Indie]",0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,Cotton,1.0,308040,72405,0,308040,Back to Bed,"[Action, Casual, Indie]",0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,Santiago ♥D,1.3,308040,294867,0,308040,Back to Bed,"[Action, Casual, Indie]",0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


* Se eliminan filas que contengan vacíos, para garantizar que la matriz pueda generarse sin errores.

In [36]:
data_reviews1.isna().sum()

username                         0
hours                            0
product_id                       0
user_id                          0
ratings                          0
id                               0
title                            0
genres                           0
Accounting                   33113
Action                       33113
Adventure                    33113
Animation &amp; Modeling     33113
Audio Production             33113
Casual                       33113
Design &amp; Illustration    33113
Early Access                 33113
Education                    33113
Free to Play                 33113
Indie                        33113
Massively Multiplayer        33113
Photo Editing                33113
RPG                          33113
Racing                       33113
Simulation                   33113
Software Training            33113
Sports                       33113
Strategy                     33113
Utilities                    33113
Video Production    

In [37]:
data_reviews1.dropna(subset=['Accounting','Action','Adventure','Animation &amp; Modeling','Audio Production','Casual','Design &amp; Illustration','Early Access','Education','Free to Play','Indie','Massively Multiplayer','Photo Editing','RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Web Publishing'], inplace=True)

In [38]:
data_reviews1.shape

(65891, 30)

* Se genera una tabla que solo contenga los géneros, esta se usará para construir la matriz.

In [39]:
data_reviews1 = data_reviews1.reset_index(drop=True)
generos = data_reviews1.drop('username',1).drop('hours',1).drop('product_id',1).drop('user_id',1).drop('ratings',1).drop('id',1).drop('title',1).drop('genres',1)

In [40]:
generos.head()

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


***Generación del perfil de usuario***

Utilizando las revisiones (*'ratings'*) y multiplicándolas dentro de la tabla de géneros, se realiza una ponderación de cada género, resultando un producto escalado entre una matriz y un vector. Para esto se usará la función *'dot'*.

In [41]:
perfil_usuario= generos.transpose().dot(data_reviews1['ratings'])

In [42]:
perfil_usuario

Accounting                       0.0
Action                       16663.0
Adventure                     8285.0
Animation &amp; Modeling       126.0
Audio Production                57.0
Casual                       10672.0
Design &amp; Illustration      414.0
Early Access                  1121.0
Education                       19.0
Free to Play                  2778.0
Indie                        17873.0
Massively Multiplayer         1612.0
Photo Editing                   86.0
RPG                           9126.0
Racing                        1556.0
Simulation                    9997.0
Software Training               65.0
Sports                         774.0
Strategy                     10834.0
Utilities                      398.0
Video Production               182.0
Web Publishing                 343.0
dtype: float64

* Después de tener el perfil del usuario, se extrae una tabla de géneros poniendo como índice el *'id'* de cada juego, esto con el fin de poder cruzar los resultados con la tabla original de juegos y traer el nombre de cada juego recomendado.

In [43]:
tabla = games_copy.reset_index(drop=True)
tabla_generos = tabla.drop('title',1).drop('genres',1)
tabla_generos.dropna(subset=['id'], inplace=True)

In [44]:
tabla_generos = tabla_generos.set_index(tabla_generos['id'])

In [45]:
tabla_generos = tabla_generos.drop('id',1)
tabla_generos.isna().sum()
tabla_generos.head()

,Accounting,Action,Adventure,Animation &amp; Modeling,Audio Production,Casual,Design &amp; Illustration,Early Access,Education,Free to Play,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
id,,,,,,,,,,,,,,,,,,,,,
761140,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
643980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
670290,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0
767400,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
772540,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0


* Para garantizar que las recomendaciones se generen sin errores, se eliminan las filas que no contengan información de los géneros.

In [46]:
tabla_generos.isna().sum()

Accounting                   3217
Action                       3217
Adventure                    3217
Animation &amp; Modeling     3217
Audio Production             3217
Casual                       3217
Design &amp; Illustration    3217
Early Access                 3217
Education                    3217
Free to Play                 3217
Indie                        3217
Massively Multiplayer        3217
Photo Editing                3217
RPG                          3217
Racing                       3217
Simulation                   3217
Software Training            3217
Sports                       3217
Strategy                     3217
Utilities                    3217
Video Production             3217
Web Publishing               3217
dtype: int64

In [47]:
tabla_generos.dropna(subset=['Accounting','Action','Adventure','Animation &amp; Modeling','Audio Production','Casual','Design &amp; Illustration','Early Access','Education','Free to Play','Indie','Massively Multiplayer','Photo Editing','RPG','Racing','Simulation','Software Training','Sports','Strategy','Utilities','Video Production','Web Publishing'], inplace=True)

In [48]:
tabla_generos.isna().sum()

Accounting                   0
Action                       0
Adventure                    0
Animation &amp; Modeling     0
Audio Production             0
Casual                       0
Design &amp; Illustration    0
Early Access                 0
Education                    0
Free to Play                 0
Indie                        0
Massively Multiplayer        0
Photo Editing                0
RPG                          0
Racing                       0
Simulation                   0
Software Training            0
Sports                       0
Strategy                     0
Utilities                    0
Video Production             0
Web Publishing               0
dtype: int64

***Creación del sistema de recomendación***

Después de tener el perfil de cada usuario y la lista completa de juegos con sus respectivos géneros, se lleva el peso promedio de cada juego basado en el perfil de ingreso para luego recomendar los primeros 20 juegos que más se adecúan a ese perfil.

In [49]:
recomendacion_table = ((tabla_generos*perfil_usuario).sum(axis=1))/(perfil_usuario.sum())
recomendacion_table.head()

id
761140    0.710242
643980    0.436767
670290    0.452716
767400    0.383089
772540    0.337940
dtype: float64

* Se ordenan las recomendaciones en orden descendente, para poner las de más peso al inicio.

In [50]:
recomendacion_table = recomendacion_table.sort_values(ascending=False)
recomendacion_table.head()

id
405350    0.969768
602510    0.939891
485950    0.939891
500590    0.926286
590630    0.926286
dtype: float64

* Finalmente se cruza con el dataset original de juegos para traer el nombre de los juegos recomendados.

In [51]:
games = data_games.loc[data_games['id'].isin(recomendacion_table.head(20).keys())]
n_games = games['title']

In [52]:
n_games

3552                           Kitty Powers' Matchmaker
3850                                   Ice Cream Surfer
4141                         Celestian Tales: Old North
4188                                        Marble Muse
8092       DROD: Smitemaster's Selection Expansions 1+2
8494                                  Lords of New York
9631          Vinnie and Spike - Awesomenauts Character
10424                                    Girls and Quiz
10474                        Gettysburg: The Tide Turns
10728                                    Fable Rush OST
12461                                     Now Man Flies
12874                         Bridge Constructor Portal
14722                                Last Toon Standing
16441                                      Mad Arkanoid
19156    Fantasy Grounds - A Wedding at Axebridge (RMC)
20871                        Yomawari: Night Alone / 夜廻
22321                         Puppy Dog: Jigsaw Puzzles
22957                              Rise of the A

***Probando el recomendador con un usuario aleatorio***

* Se muestran los juegos que un usuario específico ha jugado.

In [89]:
user = data_reviews1[(data_reviews1["user_id"] == 268687)]
user[['title', 'genres']]

,title,genres
0,Back to Bed,"[Action, Casual, Indie]"
5790,Hotline Miami 2: Wrong Number,"[Action, Indie]"
31607,Overlord™,[RPG]
37920,LYNE,"[Casual, Indie]"
48991,Hunted: The Demon’s Forge™,"[Action, Adventure]"
50976,Farming Simulator 15,[Simulation]
56200,Divinity: Original Sin (Classic),"[Adventure, Indie, RPG, Strategy]"


In [ ]:
* Se extrae la tabla de géneros del usuario.

In [90]:
tabla_2 = user.reset_index(drop=True)
tabla_user = tabla_2.drop('username',1).drop('hours',1).drop('user_id',1).drop('ratings',1).drop('id',1).drop('title',1).drop('genres',1)
tabla_user.dropna(subset=['product_id'], inplace=True)

In [91]:
tabla_user = tabla_user.set_index(tabla_user['product_id'])

In [ ]:
tabla_user = tabla_user.drop('product_id',1)

* Luego se realiza una recomendación específica para el usuario, tomando como base el perfil del usuario generado anteriormente.

In [ ]:
recomendacion_user = ((tabla_user*perfil_usuario).sum(axis=1))/(perfil_usuario.sum())
recomendacion_user = recomendacion_user.sort_values(ascending=False)
recomendacion_user.head()

* Por último se muestran los juegos recomendados para el usuario y sus respectivos géneros.

In [94]:
games_u = data_games.loc[data_games['id'].isin(recomendacion_user.head(10).keys())]
games_u[['title', 'genres']]

,title,genres
348,Overlord™,[RPG]
642,Hunted: The Demon’s Forge™,"[Action, Adventure]"
1972,LYNE,"[Casual, Indie]"
2353,Divinity: Original Sin (Classic),"[Adventure, Indie, RPG, Strategy]"
2798,Farming Simulator 15,[Simulation]
27167,Hotline Miami 2: Wrong Number,"[Action, Indie]"
28585,Back to Bed,"[Action, Casual, Indie]"


**Análisis:** En el caso de este usuario, los géneros de los juegos que normalmente juega son *Action, Casual, Indie, RPG, Adventure, Simulation, Strategy*. Se puede evidenciar como el sistema de recomendación muestra juegos que están contenidos dentro de estos géneros y no genera recomendaciones por fuera de los mismos. Es importante tener en cuenta que los géneros iniciales no tenían un orden, por lo que no es comparable el orden de recomendación.